## COMP 579 - Assignment 3 
#### Names: Marc-Antoine Nadeau (261114549) & Jessie Kurtz (26 ...)




## Question 1 

In [ ]:
import gym # type: ignore
import torch # type: ignore
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt # type: ignore

In [ ]:
ACROBOT = 'Acrobot-v1'
ALE = "ALE/Assault-ram-v5"

### Define a Q-Network

In [ ]:
class QNetwork(nn.Module):
    # Q-Network
    def __init__(self, state_dim, action_dim, hidden_dim=256, num_layers=2):
        super(QNetwork, self).__init__()
        
        layers = []
        # input layer
        layers.append(nn.Linear(state_dim, hidden_dim))
        layers.append(nn.ReLU())
        # hidden layers
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        
        # output layer
        layers.append(nn.Linear(hidden_dim, action_dim))
        # set model
        self.model = nn.Sequential(*layers)
        # initialize weights
        self.initialize_weights()
    
    # initialize weights
    def initialize_weights(self, initial_weigths=0.0001):
        for m in self.model:
            if isinstance(m, nn.Linear):
                nn.init.uniform_(m.weight, -initial_weigths, initial_weigths)
                nn.init.uniform_(m.bias, -initial_weigths, initial_weigths)
    
    # forward pass
    def forward(self, x):
        return self.model(x)

### Expected Sarsa

In [ ]:
class ExpectedSarsaAgent: 
    def __init__(self,)

### Q-Learning

## Question 2

## Question 3